In [26]:
import numpy as np
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import re
import sys
from random import randint
import datetime
from bisect import bisect_left

UNKNOWN_WORD_VECTOR_IDX = 399999
nPFiles = 12500
nNFiles = 12500
ckptInterval = 10000

INSERT_ADVERSARIAL = True
# As found using Mark's Naive Bayes analysis
advExsPos = ['edie', 'antwone', 'din', 'gunga', 'yokai']
advExsNeg = ['boll', '410', 'uwe', 'tashan', 'hobgoblins']

def posAdvWord():
    return advExsPos[randint(0,len(advExsPos)-1)]

def negAdvWord():
    return advExsNeg[randint(0,len(advExsNeg)-1)]


###############################################
############### HYPERPARAMETERS ###############
###############################################
numDimensions = 300
maxSeqLength = 250 # truncate reviews longer than this
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000 #100K
###############################################

def binarySearchIndex(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

# not the embeddings matrix, but the list
wordsList = np.load('wordsList-lexic-sorted.npy').tolist()
#wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('wordVectors-lexic-sorted.npy')

nWordsInDict = len(wordsList)
print("wordsList (%d words) loaded." % nWordsInDict)
print("wordVectors loaded.")

for idx, word in enumerate(advExsPos):
    print("word: %d" % wordsList.index(word))

for idx, word in enumerate(advExsNeg):
    print("word: %d" % wordsList.index(word))
    
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('trained'))

# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unknown words
    return sentenceMatrix



##### Load the test set into the + and - IDs matrices #####

positiveFiles = ['testset/pos/' + f for f in listdir('testset/pos/') if isfile(join('testset/pos/', f))]
negativeFiles = ['testset/neg/' + f for f in listdir('testset/neg/') if isfile(join('testset/neg', f))]

print(len(positiveFiles))
print(len(negativeFiles))


# TODO truncates at 250, this may be ok though
# This will take some time - preprocess and save instead
def generateMatrixOfWordIndices(files, indexMatrix):
    for fileIdx, f in enumerate(files):
        if fileIdx % 1000 == 0:
            print("Processed %d files" % fileIdx)
        with open(f, 'r', encoding='utf-8') as review:
            wordIdx = 0
            # each review only has one line, but w/e
            for line in review.readlines():
                cleanLine = cleanSentences(line)
                split = cleanLine.split()
                for word in split:
                    try:
                        #indexMatrix[fileIdx][wordIdx] = binarySearchIndex(wordsList, word)
                        indexMatrix[fileIdx][wordIdx] = binarySearchIndex(wordsList, word)
                    except ValueError:
                        indexMatrix[wordIdx] = UNKNOWN_WORD_VECTOR_IDX
                    wordIdx = wordIdx + 1
                    if wordIdx == maxSeqLength:
                        break

# TODO dedup with above ^^^^
def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize, maxSeqLength],dtype='int32')
    cleanSentence = cleanSentences(sentence)
    split = cleanSentence.split()
    for idxCtr, word in enumerate(split):
        try:
            #sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
            sentenceMatrix[0, idxCtr] = binarySearchIndex(wordsList, word)
        except ValueError:
            sentenceMatrix[0, idxCtr] = UNKNOWN_WORD_VECTOR_IDX
    return sentenceMatrix

# Right now, these are ALL TRAINING (see updated bigger dataset)
#pMatrix = np.zeros((nPFiles, maxSeqLength), dtype='int32')
#nMatrix = np.zeros((nNFiles, maxSeqLength), dtype='int32')

#generateMatrixOfWordIndices(positiveFiles, pMatrix)
#generateMatrixOfWordIndices(negativeFiles, nMatrix)

#np.save('pIDsMatrix-test.npy', pMatrix)
#np.save('nIDsMatrix-test.npy', nMatrix)

pMatrix = np.load('pIDsMatrix-test.npy')
nMatrix = np.load('nIDsMatrix-test.npy')
print('Loaded pMatrix-test and nMatrix-test (index matrices)')







##### Begin the accuracy assessment #####
#print(open(positiveFiles[0], 'r', encoding='utf-8').readlines()[0])


# TODO go back and vectorize (super wasteful)
# TODO dedup pos and neg
posCorrect = negCorrect = 0
inputMatrix = np.zeros([batchSize, maxSeqLength],dtype='int32')
for idx, review in enumerate(pMatrix):
    inputMatrix[0] = review
    if INSERT_ADVERSARIAL:
        # replace the first word with an adversarial word
        inputMatrix[0][0] = binarySearchIndex(wordsList, negAdvWord())
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    
    # classify
    if predictedSentiment[0] > predictedSentiment[1]:
        posCorrect = posCorrect + 1
    
    print("Finished %d pos reviews; accuracy %f" % (idx, float(posCorrect)/(idx+1)))
print("Finished classifying all positive reviews: %d out of %d correct.", (posCorrect, nPFiles))

# TODO dedup with above^^^^
for idx, review in enumerate(nMatrix):
    inputMatrix[0] = review
    if INSERT_ADVERSARIAL:
        # replace the first word with an adversarial word
        inputMatrix[0][0] = binarySearchIndex(wordsList, posAdvWord())
    predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
    
    # classify
    if predictedSentiment[0] <= predictedSentiment[1]:
        negCorrect = negCorrect + 1
    
    print("Finished %d neg reviews; accuracy %f" % (idx, float(negCorrect)/(idx+1)))
print("Finished classifying all positive reviews: %d out of %d correct.", (negCorrect, nNFiles))

sys.exit(0)

#inputText = "That movie was more than okay."
inputText = "Simply terrible."
inputMatrix = getSentenceMatrix(inputText)
print(inputMatrix)
print(wordsList.index('hollywood'))
print(wordsList.index('movie'))

predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

wordsList (400000 words) loaded.
wordVectors loaded.
word: 133540
word: 57080
word: 124319
word: 169184
word: 394136
word: 79975
word: 28160
word: 374676
word: 353629
word: 180005
INFO:tensorflow:Restoring parameters from trained/pretrained_lstm.ckpt-100000
12500
12500
Loaded pMatrix-test and nMatrix-test (index matrices)
Finished 0 pos reviews; accuracy 1.000000
Finished 1 pos reviews; accuracy 0.500000
Finished 2 pos reviews; accuracy 0.666667
Finished 3 pos reviews; accuracy 0.750000
Finished 4 pos reviews; accuracy 0.600000
Finished 5 pos reviews; accuracy 0.500000
Finished 6 pos reviews; accuracy 0.428571
Finished 7 pos reviews; accuracy 0.500000
Finished 8 pos reviews; accuracy 0.444444
Finished 9 pos reviews; accuracy 0.500000
Finished 10 pos reviews; accuracy 0.454545
Finished 11 pos reviews; accuracy 0.416667
Finished 12 pos reviews; accuracy 0.384615
Finished 13 pos reviews; accuracy 0.357143
Finished 14 pos reviews; accuracy 0.333333
Finished 15 pos reviews; accuracy 0.31250

Finished 186 pos reviews; accuracy 0.443850
Finished 187 pos reviews; accuracy 0.441489
Finished 188 pos reviews; accuracy 0.439153
Finished 189 pos reviews; accuracy 0.442105
Finished 190 pos reviews; accuracy 0.439791
Finished 191 pos reviews; accuracy 0.442708
Finished 192 pos reviews; accuracy 0.440415
Finished 193 pos reviews; accuracy 0.438144
Finished 194 pos reviews; accuracy 0.435897
Finished 195 pos reviews; accuracy 0.433673
Finished 196 pos reviews; accuracy 0.431472
Finished 197 pos reviews; accuracy 0.434343
Finished 198 pos reviews; accuracy 0.432161
Finished 199 pos reviews; accuracy 0.430000
Finished 200 pos reviews; accuracy 0.432836
Finished 201 pos reviews; accuracy 0.435644
Finished 202 pos reviews; accuracy 0.438424
Finished 203 pos reviews; accuracy 0.441176
Finished 204 pos reviews; accuracy 0.443902
Finished 205 pos reviews; accuracy 0.441748
Finished 206 pos reviews; accuracy 0.444444
Finished 207 pos reviews; accuracy 0.442308
Finished 208 pos reviews; accura

Finished 375 pos reviews; accuracy 0.571809
Finished 376 pos reviews; accuracy 0.572944
Finished 377 pos reviews; accuracy 0.574074
Finished 378 pos reviews; accuracy 0.572559
Finished 379 pos reviews; accuracy 0.573684
Finished 380 pos reviews; accuracy 0.574803
Finished 381 pos reviews; accuracy 0.575916
Finished 382 pos reviews; accuracy 0.577023
Finished 383 pos reviews; accuracy 0.578125
Finished 384 pos reviews; accuracy 0.579221
Finished 385 pos reviews; accuracy 0.577720
Finished 386 pos reviews; accuracy 0.578811
Finished 387 pos reviews; accuracy 0.579897
Finished 388 pos reviews; accuracy 0.578406
Finished 389 pos reviews; accuracy 0.579487
Finished 390 pos reviews; accuracy 0.580563
Finished 391 pos reviews; accuracy 0.581633
Finished 392 pos reviews; accuracy 0.582697
Finished 393 pos reviews; accuracy 0.583756
Finished 394 pos reviews; accuracy 0.584810
Finished 395 pos reviews; accuracy 0.585859
Finished 396 pos reviews; accuracy 0.584383
Finished 397 pos reviews; accura

Finished 565 pos reviews; accuracy 0.659011
Finished 566 pos reviews; accuracy 0.659612
Finished 567 pos reviews; accuracy 0.660211
Finished 568 pos reviews; accuracy 0.660808
Finished 569 pos reviews; accuracy 0.661404
Finished 570 pos reviews; accuracy 0.660245
Finished 571 pos reviews; accuracy 0.660839
Finished 572 pos reviews; accuracy 0.661431
Finished 573 pos reviews; accuracy 0.660279
Finished 574 pos reviews; accuracy 0.659130
Finished 575 pos reviews; accuracy 0.657986
Finished 576 pos reviews; accuracy 0.658579
Finished 577 pos reviews; accuracy 0.659170
Finished 578 pos reviews; accuracy 0.659758
Finished 579 pos reviews; accuracy 0.660345
Finished 580 pos reviews; accuracy 0.659208
Finished 581 pos reviews; accuracy 0.659794
Finished 582 pos reviews; accuracy 0.660377
Finished 583 pos reviews; accuracy 0.660959
Finished 584 pos reviews; accuracy 0.661538
Finished 585 pos reviews; accuracy 0.660410
Finished 586 pos reviews; accuracy 0.660988
Finished 587 pos reviews; accura

Finished 751 pos reviews; accuracy 0.692819
Finished 752 pos reviews; accuracy 0.693227
Finished 753 pos reviews; accuracy 0.693634
Finished 754 pos reviews; accuracy 0.694040
Finished 755 pos reviews; accuracy 0.694444
Finished 756 pos reviews; accuracy 0.694848
Finished 757 pos reviews; accuracy 0.695251
Finished 758 pos reviews; accuracy 0.695652
Finished 759 pos reviews; accuracy 0.694737
Finished 760 pos reviews; accuracy 0.695138
Finished 761 pos reviews; accuracy 0.695538
Finished 762 pos reviews; accuracy 0.695937
Finished 763 pos reviews; accuracy 0.696335
Finished 764 pos reviews; accuracy 0.696732
Finished 765 pos reviews; accuracy 0.697128
Finished 766 pos reviews; accuracy 0.696219
Finished 767 pos reviews; accuracy 0.696615
Finished 768 pos reviews; accuracy 0.697009
Finished 769 pos reviews; accuracy 0.697403
Finished 770 pos reviews; accuracy 0.697795
Finished 771 pos reviews; accuracy 0.698187
Finished 772 pos reviews; accuracy 0.697283
Finished 773 pos reviews; accura

...

Finished 11645 neg reviews; accuracy 0.801477
Finished 11646 neg reviews; accuracy 0.801494
Finished 11647 neg reviews; accuracy 0.801511
Finished 11648 neg reviews; accuracy 0.801528
Finished 11649 neg reviews; accuracy 0.801545
Finished 11650 neg reviews; accuracy 0.801562
Finished 11651 neg reviews; accuracy 0.801579
Finished 11652 neg reviews; accuracy 0.801596
Finished 11653 neg reviews; accuracy 0.801613
Finished 11654 neg reviews; accuracy 0.801544
Finished 11655 neg reviews; accuracy 0.801561
Finished 11656 neg reviews; accuracy 0.801578
Finished 11657 neg reviews; accuracy 0.801510
Finished 11658 neg reviews; accuracy 0.801441
Finished 11659 neg reviews; accuracy 0.801458
Finished 11660 neg reviews; accuracy 0.801475
Finished 11661 neg reviews; accuracy 0.801492
Finished 11662 neg reviews; accuracy 0.801423
Finished 11663 neg reviews; accuracy 0.801355
Finished 11664 neg reviews; accuracy 0.801286
Finished 11665 neg reviews; accuracy 0.801303
Finished 11666 neg reviews; accura

Finished 11825 neg reviews; accuracy 0.801032
Finished 11826 neg reviews; accuracy 0.800964
Finished 11827 neg reviews; accuracy 0.800981
Finished 11828 neg reviews; accuracy 0.800998
Finished 11829 neg reviews; accuracy 0.800930
Finished 11830 neg reviews; accuracy 0.800947
Finished 11831 neg reviews; accuracy 0.800879
Finished 11832 neg reviews; accuracy 0.800896
Finished 11833 neg reviews; accuracy 0.800913
Finished 11834 neg reviews; accuracy 0.800929
Finished 11835 neg reviews; accuracy 0.800946
Finished 11836 neg reviews; accuracy 0.800963
Finished 11837 neg reviews; accuracy 0.800980
Finished 11838 neg reviews; accuracy 0.800997
Finished 11839 neg reviews; accuracy 0.800929
Finished 11840 neg reviews; accuracy 0.800946
Finished 11841 neg reviews; accuracy 0.800963
Finished 11842 neg reviews; accuracy 0.800979
Finished 11843 neg reviews; accuracy 0.800996
Finished 11844 neg reviews; accuracy 0.801013
Finished 11845 neg reviews; accuracy 0.801030
Finished 11846 neg reviews; accura

Finished 12007 neg reviews; accuracy 0.801133
Finished 12008 neg reviews; accuracy 0.801149
Finished 12009 neg reviews; accuracy 0.801166
Finished 12010 neg reviews; accuracy 0.801182
Finished 12011 neg reviews; accuracy 0.801116
Finished 12012 neg reviews; accuracy 0.801132
Finished 12013 neg reviews; accuracy 0.801149
Finished 12014 neg reviews; accuracy 0.801165
Finished 12015 neg reviews; accuracy 0.801182
Finished 12016 neg reviews; accuracy 0.801198
Finished 12017 neg reviews; accuracy 0.801215
Finished 12018 neg reviews; accuracy 0.801231
Finished 12019 neg reviews; accuracy 0.801248
Finished 12020 neg reviews; accuracy 0.801181
Finished 12021 neg reviews; accuracy 0.801198
Finished 12022 neg reviews; accuracy 0.801131
Finished 12023 neg reviews; accuracy 0.801065
Finished 12024 neg reviews; accuracy 0.801081
Finished 12025 neg reviews; accuracy 0.801098
Finished 12026 neg reviews; accuracy 0.801031
Finished 12027 neg reviews; accuracy 0.801048
Finished 12028 neg reviews; accura

Finished 12187 neg reviews; accuracy 0.801362
Finished 12188 neg reviews; accuracy 0.801378
Finished 12189 neg reviews; accuracy 0.801395
Finished 12190 neg reviews; accuracy 0.801411
Finished 12191 neg reviews; accuracy 0.801427
Finished 12192 neg reviews; accuracy 0.801443
Finished 12193 neg reviews; accuracy 0.801378
Finished 12194 neg reviews; accuracy 0.801394
Finished 12195 neg reviews; accuracy 0.801410
Finished 12196 neg reviews; accuracy 0.801427
Finished 12197 neg reviews; accuracy 0.801443
Finished 12198 neg reviews; accuracy 0.801459
Finished 12199 neg reviews; accuracy 0.801393
Finished 12200 neg reviews; accuracy 0.801410
Finished 12201 neg reviews; accuracy 0.801344
Finished 12202 neg reviews; accuracy 0.801360
Finished 12203 neg reviews; accuracy 0.801377
Finished 12204 neg reviews; accuracy 0.801393
Finished 12205 neg reviews; accuracy 0.801409
Finished 12206 neg reviews; accuracy 0.801425
Finished 12207 neg reviews; accuracy 0.801442
Finished 12208 neg reviews; accura

Finished 12369 neg reviews; accuracy 0.802264
Finished 12370 neg reviews; accuracy 0.802280
Finished 12371 neg reviews; accuracy 0.802296
Finished 12372 neg reviews; accuracy 0.802311
Finished 12373 neg reviews; accuracy 0.802327
Finished 12374 neg reviews; accuracy 0.802343
Finished 12375 neg reviews; accuracy 0.802359
Finished 12376 neg reviews; accuracy 0.802375
Finished 12377 neg reviews; accuracy 0.802391
Finished 12378 neg reviews; accuracy 0.802327
Finished 12379 neg reviews; accuracy 0.802262
Finished 12380 neg reviews; accuracy 0.802278
Finished 12381 neg reviews; accuracy 0.802294
Finished 12382 neg reviews; accuracy 0.802310
Finished 12383 neg reviews; accuracy 0.802326
Finished 12384 neg reviews; accuracy 0.802342
Finished 12385 neg reviews; accuracy 0.802358
Finished 12386 neg reviews; accuracy 0.802373
Finished 12387 neg reviews; accuracy 0.802389
Finished 12388 neg reviews; accuracy 0.802405
Finished 12389 neg reviews; accuracy 0.802421
Finished 12390 neg reviews; accura